In [3]:
import pandas as pd
from pathlib import Path
from functools import reduce
import numpy as np

ruta_archivo = Path(r'C:\Users\agusm\Trabajo-Final-Lab-Datos-\Archivos_xlsx\Planta_2023.xlsx')
fecha_limite_str = '2023-03-06'  # usa ISO para evitar ambigüedad

def preparar_hoja(df: pd.DataFrame, nombre_hoja: str) -> pd.DataFrame:
    df = df.copy()
    
    # Construir FECHA_HORA
    if 'FECHA_HORA' in df.columns:
        ts = pd.to_datetime(df['FECHA_HORA'], errors='coerce')
    elif {'DIA', 'HORA'}.issubset(df.columns):

        df['HORA'] = df['HORA'].astype(str).str.extract(r'(\d{1,2}:\d{2}:\d{2})')[0]
        ts = pd.to_datetime(df['DIA'].astype(str) + ' ' + df['HORA'].astype(str), errors='coerce')
    elif 'DIA' in df.columns:
        ts = pd.to_datetime(df['DIA'], errors='coerce')
    else:
        return None

    df['FECHA_HORA'] = ts
    #quiero saber si hay nans
    print(f"Hoja {nombre_hoja}: Nulos en FECHA_HORA antes de dropna: {df['FECHA_HORA'].isna().sum()}")
    dias_nulos = df['DIA'][df['FECHA_HORA'].isna()]
    horas_nulas = df['HORA'][df['FECHA_HORA'].isna()]
    if not dias_nulos.empty or not horas_nulas.empty:
        print(f"Días nulos:\n{dias_nulos}")
        print(f"Horas nulas:\n{horas_nulas}") 
    df = df.dropna(subset=['FECHA_HORA']).reset_index(drop=True)
    # 1) Normalizar a una fila por timestamp dentro de la hoja
    #    - numéricas: 'mean' (si tus columnas son intensidades; usa 'sum' si son totales)
    #    - no numéricas: 'first'
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    other_cols = [c for c in df.columns if c not in numeric_cols + ['DIA', 'HORA', 'FECHA_HORA']]
    print(f"Duplicadas por FECHA_HORA en hoja {nombre_hoja} antes de procesmiento: {df.duplicated(subset=['FECHA_HORA']).sum()}")

    agg_map = {**{c: 'mean' for c in numeric_cols}, **{c: 'first' for c in other_cols}}
    df = (
        df.drop(columns=['DIA', 'HORA'], errors='ignore')
          .groupby('FECHA_HORA', as_index=False)
          .agg(agg_map)
    )
    print(f"Duplicadas por FECHA_HORA en hoja {nombre_hoja} despues de procesmiento: {df.duplicated(subset=['FECHA_HORA']).sum()}")
    #quiero mostrar las filas duplicadas
    filas_duplicadas = df[df.duplicated(subset=['FECHA_HORA'], keep=False)]
    if not filas_duplicadas.empty:
        print(f"Filas duplicadas por FECHA_HORA en hoja {nombre_hoja}:\n{filas_duplicadas}")
        print(filas_duplicadas)

    
    return df

# Carga y preparación por hoja
dict_de_hojas = pd.read_excel(ruta_archivo, sheet_name=None)
#Solo quiero seleccionar las hojas cuyo nombre empiece con "Consolidado"
dict_de_hojas = {nombre: df for nombre, df in dict_de_hojas.items() if nombre.startswith("Consolidado")}
hojas_preparadas = [h for nombre, df in dict_de_hojas.items()
                    if (h := preparar_hoja(df, nombre)) is not None and len(h) > 0]

# Merge por FECHA_HORA (ya sin duplicados por hoja)
df_combinado = reduce(lambda l, r: pd.merge(l, r, on='FECHA_HORA', how='outer'), hojas_preparadas)
df_combinado = df_combinado.sort_values('FECHA_HORA').reset_index(drop=True)

# Diagnóstico duplicados tras el merge (debería bajar muchísimo)
print("Duplicadas exactas:", df_combinado.duplicated().sum())
print("Duplicadas por FECHA_HORA:", df_combinado.duplicated(subset=['FECHA_HORA']).sum())



# Filtro de fecha (elige 'desde' o 'hasta' explícitamente)
fecha_limite = pd.to_datetime(fecha_limite_str)

# Caso A: quedarme con datos HASTA esa fecha (incluida)
df_filtrado = df_combinado[df_combinado['FECHA_HORA'] >= fecha_limite].copy()

# Caso B: quedarme con datos DESDE esa fecha (excluida)
# df_filtrado = df_combinado[df_combinado['FECHA_HORA'] > fecha_limite].copy()

print(f"Rango: {df_filtrado['FECHA_HORA'].min()} -> {df_filtrado['FECHA_HORA'].max()}")
print(f"Filas: {len(df_filtrado)}, Columnas: {df_filtrado.shape[1]}")


Hoja Consolidado KPI: Nulos en FECHA_HORA antes de dropna: 1
Días nulos:
12009   NaT
Name: DIA, dtype: datetime64[ns]
Horas nulas:
12009    NaN
Name: HORA, dtype: object
Duplicadas por FECHA_HORA en hoja Consolidado KPI antes de procesmiento: 199
Duplicadas por FECHA_HORA en hoja Consolidado KPI despues de procesmiento: 0
Hoja Consolidado Produccion: Nulos en FECHA_HORA antes de dropna: 1
Días nulos:
12009   NaT
Name: DIA, dtype: datetime64[ns]
Horas nulas:
12009    NaN
Name: HORA, dtype: object
Duplicadas por FECHA_HORA en hoja Consolidado Produccion antes de procesmiento: 199
Duplicadas por FECHA_HORA en hoja Consolidado Produccion despues de procesmiento: 0
Hoja Consolidado EE: Nulos en FECHA_HORA antes de dropna: 2
Días nulos:
12009   NaT
12010   NaT
Name: DIA, dtype: datetime64[ns]
Horas nulas:
12009    NaN
12010    NaN
Name: HORA, dtype: object
Duplicadas por FECHA_HORA en hoja Consolidado EE antes de procesmiento: 195
Duplicadas por FECHA_HORA en hoja Consolidado EE despues de p

In [4]:
dias_raw   = pd.DatetimeIndex(df_filtrado['FECHA_HORA'].dt.normalize().unique()).sort_values()
inicio = dias_raw.min()
fin    = dias_raw.max()
full   = pd.date_range(inicio, fin, freq='D')

# Faltan en el raw (no hay ningún timestamp ese día)
faltan_en_raw = full.difference(dias_raw)

print(f"Días en raw: {len(dias_raw)}")
print(f"Días desde {inicio.date()} hasta {fin.date()}: {len(full)}")
print(f"Días esperados: {len(full)}")
print(f"Faltan en raw (todas las hojas): {len(faltan_en_raw)}")
#cuales son esos dias
print(faltan_en_raw)

Días en raw: 231
Días desde 2023-03-06 hasta 2023-10-26: 235
Días esperados: 235
Faltan en raw (todas las hojas): 4
DatetimeIndex(['2023-03-31', '2023-05-31', '2023-08-30', '2023-08-31'], dtype='datetime64[ns]', freq=None)


In [5]:
df_filtrado 

,FECHA_HORA,EE Planta / Hl,EE Elaboracion / Hl,EE Bodega / Hl,EE Cocina / Hl,EE Envasado / Hl,EE Linea 2 / Hl,EE Linea 3 / Hl,EE Linea 4 / Hl,EE Linea 5 / Hl,...,Aire Elaboracion (m3),Aire Envasado (M3),Aire Servicios (M3),Tot Aire Expulsion,Totalizador_Aire_Bodega,Totalizador_Aire_Cocina,Totalizador_Aire_L2,Totalizador_Aire_L3,Totaliador_Aire_L4,Totalizador_Aire_L5
6066,2023-03-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00000,0.0000,0.00000,0.0,0.0000,0.00000,0.0000,0.00,0.00,0.00
6067,2023-03-06 01:00:00,8.003075,0.683230,0.670807,NaN,2.624224,1.547560,1.589623,1.367424,NaN,...,845.37000,739.4000,464.23000,0.0,727.7500,117.62000,211.1000,292.61,164.47,71.22
6068,2023-03-06 02:00:00,8.250129,0.699515,0.704362,NaN,2.701131,1.641601,1.659204,1.362264,NaN,...,1664.08000,1464.8400,931.08000,0.0,1423.5800,240.50000,411.9100,581.06,329.37,142.50
6069,2023-03-06 03:00:00,8.754102,0.725377,0.734647,NaN,2.793743,1.903045,1.644518,1.412533,NaN,...,2477.30000,2173.0400,1406.66000,0.0,2112.2200,365.08000,593.0800,873.63,492.40,213.93
6070,2023-03-06 04:00:00,9.109177,0.745027,0.754973,NaN,2.921338,2.044022,1.637267,1.511088,NaN,...,3261.23000,2910.8400,1868.93000,0.0,2772.2800,488.95000,804.2600,1171.28,649.89,285.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11810,2023-10-26 03:00:00,19.366571,2.040100,2.325815,NaN,3.258145,2.316117,NaN,NaN,NaN,...,993.28125,971.8125,762.90625,0.0,675.5625,317.71875,971.8125,0.00,0.00,0.00
11811,2023-10-26 04:00:00,20.934877,2.161290,2.455645,NaN,3.431452,2.446269,NaN,NaN,NaN,...,1325.84375,1235.5625,1068.59375,0.0,900.6250,425.21875,1235.5625,0.00,0.00,0.00
11812,2023-10-26 05:00:00,20.861716,2.095390,2.391097,NaN,3.405405,2.429268,NaN,NaN,NaN,...,1895.34375,1603.7500,1279.90625,0.0,1379.8125,515.53125,1603.7500,0.00,0.00,0.00
11813,2023-10-26 06:00:00,20.964980,2.021419,2.321285,NaN,3.394913,2.432289,NaN,NaN,NaN,...,2227.81250,1944.1875,1523.00000,0.0,1619.3750,608.43750,1944.1875,0.00,0.00,0.00


In [6]:
import pandas as pd

# Agrupa por la fecha (la parte de día de 'FECHA_HORA') y encuentra la hora máxima (idxmax) para esa fecha.
indices_ultima_medicion = df_filtrado.groupby(
    df_filtrado['FECHA_HORA'].dt.date
)['FECHA_HORA'].idxmax()

# --- PASO 3: Filtrar el DataFrame Original ---
# Selecciona solo las filas correspondientes a los índices de la última medición.
df_filtrado = df_filtrado.loc[indices_ultima_medicion]



df_filtrado

,FECHA_HORA,EE Planta / Hl,EE Elaboracion / Hl,EE Bodega / Hl,EE Cocina / Hl,EE Envasado / Hl,EE Linea 2 / Hl,EE Linea 3 / Hl,EE Linea 4 / Hl,EE Linea 5 / Hl,...,Aire Elaboracion (m3),Aire Envasado (M3),Aire Servicios (M3),Tot Aire Expulsion,Totalizador_Aire_Bodega,Totalizador_Aire_Cocina,Totalizador_Aire_L2,Totalizador_Aire_L3,Totaliador_Aire_L4,Totalizador_Aire_L5
6090,2023-03-06 23:59:00,9.270648,0.755990,0.761753,NaN,2.940552,1.990216,1.857971,1.441601,NaN,...,17980.670000,19027.600000,11211.730000,0.000000,14932.1000,3048.57000,5287.5900,7046.950,4010.830000,2682.230000
6115,2023-03-07 23:59:00,7.248431,0.734068,0.633494,0.524162,2.256563,2.260038,1.778930,1.508284,NaN,...,20211.180000,16197.650000,16122.170000,350.440000,15897.5500,3963.19000,2843.0200,6785.150,3946.420000,2623.060000
6140,2023-03-08 23:59:00,5.375164,0.642791,0.474405,0.315789,1.520617,4.016500,1.642118,1.482931,NaN,...,23018.780000,17072.430000,22648.790000,1308.030000,17031.7900,4678.96000,3858.5500,7110.510,3926.030000,2177.340000
6165,2023-03-09 23:59:00,7.926468,1.237505,0.963233,0.311111,0.871392,6.929725,2.383296,4.564576,NaN,...,23192.150000,6871.340000,17721.510000,1205.160000,17344.3600,4642.63000,2072.7900,3646.040,822.840000,329.670000
6190,2023-03-10 23:59:00,6.102117,0.943624,0.729104,0.272416,0.146793,17.722895,NaN,NaN,NaN,...,18879.630000,2880.240000,15481.130000,1266.300000,13279.3000,4334.03000,3.8800,1878.180,104.400000,893.780000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11731,2023-10-22 23:59:00,7.274196,0.768547,0.669597,0.348668,0.662392,3.728423,NaN,NaN,NaN,...,10510.859375,6064.859375,9558.281250,485.421875,7086.3125,2939.12500,5989.6250,68.625,0.000000,6.609375
11756,2023-10-23 23:59:00,58.805087,4.777436,5.911795,NaN,6.849231,5.644595,NaN,NaN,NaN,...,9327.734375,6851.625000,6702.640625,135.109375,6964.3750,2228.25000,6835.6875,9.125,1.109375,5.703125
11781,2023-10-24 23:59:00,33.526177,3.299732,3.651475,NaN,5.912064,3.721534,NaN,NaN,NaN,...,9987.031250,6664.765625,5228.203125,0.000000,7709.0625,2277.96875,6593.4375,70.625,0.000000,0.703125
11806,2023-10-25 23:59:00,24.449956,2.560321,2.942685,NaN,3.451703,2.730589,NaN,NaN,NaN,...,8867.742188,8678.515625,5552.742188,18.242188,6522.3750,2327.12500,8671.3125,5.750,1.453125,0.000000


In [7]:
import numpy as np
df_final = df_filtrado.copy()
df_final["Frio_diff1_lag1"] = df_final["Frio (Kw)"].astype(float).diff().shift(1)
df_final["Frio_diff7_lag1"] = df_final["Frio (Kw)"].astype(float).diff().shift(1)
roll_windows = [3, 7, 14,28]
for window in roll_windows:
    df_final[f"Frio_roll_mean_{window}_lag1"] = df_final["Frio (Kw)"].astype(float).shift(1).rolling(window=window).mean()
    df_final[f"Frio_roll_std_{window}_lag1"] = df_final["Frio (Kw)"].astype(float).shift(1).rolling(window=window).std()

#Ademas vamos a definir un z score para la columna Frio (Kw)


y = df_final["Frio (Kw)"].shift(-1)  # predecir el siguiente valor de Frio (Kw)

#ahora quiero guardar df_final en un archivo csv e y en otro archivo csv
df_final.to_csv(r'C:\Users\agusm\Trabajo-Final-Lab-Datos-\df_final.csv', index=False)
y.to_csv(r'C:\Users\agusm\Trabajo-Final-Lab-Datos-\y.csv', index=False)
